<a href="https://colab.research.google.com/github/tomoya-suzuki-1205/keiba/blob/main/%E7%AB%B6%E9%A6%ACAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 目標は回収率１００％超えにすること。
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from itertools import cycle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [3]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        547M         10G        936K        1.9G         11G
Swap:            0B          0B          0B


In [4]:
# データの読み込み
# パスは適宜変更してください
keiba_data = pd.read_csv('/content/drive/MyDrive/競馬.csv', encoding = "shift-jis")

In [5]:
keiba_data

,Unnamed: 0,age,c_weight,course,date,field,gender,head_count,horse_name,j_weight,jackie,odds,popu,race,race_name,rank,trainerA,trainerB,weight,year
0,20080101010100,2,0.0,芝1500m,8/16(土),札幌,牝,14頭,ディアジーナ,54.0,四位,4.3,2.0,1R,２歳未勝利,1,美浦,田村,468.0,2008.0
1,20080101010101,2,0.0,芝1500m,8/16(土),札幌,牡,14頭,アイディンパワー,54.0,藤田,5.2,4.0,1R,２歳未勝利,2,栗東,本田,460.0,2008.0
2,20080101010102,2,2.0,芝1500m,8/16(土),札幌,牡,14頭,ウインリーズン,54.0,安藤勝,3.1,1.0,1R,２歳未勝利,3,美浦,加藤征,442.0,2008.0
3,20080101010103,2,-4.0,芝1500m,8/16(土),札幌,牡,14頭,ヴァンエボン,54.0,小林徹,23.0,8.0,1R,２歳未勝利,4,栗東,山内,482.0,2008.0
4,20080101010104,2,-20.0,芝1500m,8/16(土),札幌,牝,14頭,ラヴェリータ,54.0,長谷川,21.9,6.0,1R,２歳未勝利,5,栗東,松元,484.0,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535589,20181002101211,4,-2.0,ダ1700m,8/26(日),小倉,牝,16頭,クリノゼノビア,55.0,小牧,243.4,16.0,12R,３歳上５００万下,12,栗東,田所,424.0,2018.0
535590,20181002101212,3,2.0,ダ1700m,8/26(日),小倉,牝,16頭,ウエスタンヒューズ,52.0,和田,69.2,12.0,12R,３歳上５００万下,13,栗東,西園,472.0,2018.0
535591,20181002101213,4,0.0,ダ1700m,8/26(日),小倉,牝,16頭,モンロー,55.0,太宰,39.5,9.0,12R,３歳上５００万下,14,栗東,飯田祐,444.0,2018.0
535592,20181002101214,4,-2.0,ダ1700m,8/26(日),小倉,牝,16頭,ディヴァインハイツ,53.0,△川又,27.5,7.0,12R,３歳上５００万下,15,栗東,石坂正,468.0,2018.0


In [6]:
# popuはpopularで人気のこと。
# oddsとはオッズのこと。馬にはそれぞれオッズが設定されており、人気な馬ほどオッズは小さい。例えば１００円賭けてオッズが6.0なら600円の払い戻し。

In [7]:
# データの読み込み
# Unnamed0のカラムは日付、レース番号、馬番号なのでrenameして分割。

keiba_data.rename(columns={"Unnamed: 0":"date_num"},inplace=True)
keiba_data["date_num"]=keiba_data["date_num"].astype(str)
keiba_data["race_num"]=keiba_data["date_num"].str[0:12].astype(int)
keiba_data["horse_num"]=keiba_data["date_num"].str[12:14].astype(int)
keiba_data.drop(columns=["date_num"],inplace=True)
# 扱いやすいようにrace_numとhorse_numは一番左に配置。
keiba_data=keiba_data.reindex(columns=["race_num","horse_num",'age', 'c_weight', 'course', 'date', 'field', 'gender', 'head_count',
       'horse_name', 'j_weight', 'jackie', 'odds', 'popu', 'race', 'race_name',
       'rank', 'trainerA', 'trainerB', 'weight', 'year'])

In [8]:
keiba_data

,race_num,horse_num,age,c_weight,course,date,field,gender,head_count,horse_name,j_weight,jackie,odds,popu,race,race_name,rank,trainerA,trainerB,weight,year
0,200801010101,0,2,0.0,芝1500m,8/16(土),札幌,牝,14頭,ディアジーナ,54.0,四位,4.3,2.0,1R,２歳未勝利,1,美浦,田村,468.0,2008.0
1,200801010101,1,2,0.0,芝1500m,8/16(土),札幌,牡,14頭,アイディンパワー,54.0,藤田,5.2,4.0,1R,２歳未勝利,2,栗東,本田,460.0,2008.0
2,200801010101,2,2,2.0,芝1500m,8/16(土),札幌,牡,14頭,ウインリーズン,54.0,安藤勝,3.1,1.0,1R,２歳未勝利,3,美浦,加藤征,442.0,2008.0
3,200801010101,3,2,-4.0,芝1500m,8/16(土),札幌,牡,14頭,ヴァンエボン,54.0,小林徹,23.0,8.0,1R,２歳未勝利,4,栗東,山内,482.0,2008.0
4,200801010101,4,2,-20.0,芝1500m,8/16(土),札幌,牝,14頭,ラヴェリータ,54.0,長谷川,21.9,6.0,1R,２歳未勝利,5,栗東,松元,484.0,2008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535589,201810021012,11,4,-2.0,ダ1700m,8/26(日),小倉,牝,16頭,クリノゼノビア,55.0,小牧,243.4,16.0,12R,３歳上５００万下,12,栗東,田所,424.0,2018.0
535590,201810021012,12,3,2.0,ダ1700m,8/26(日),小倉,牝,16頭,ウエスタンヒューズ,52.0,和田,69.2,12.0,12R,３歳上５００万下,13,栗東,西園,472.0,2018.0
535591,201810021012,13,4,0.0,ダ1700m,8/26(日),小倉,牝,16頭,モンロー,55.0,太宰,39.5,9.0,12R,３歳上５００万下,14,栗東,飯田祐,444.0,2018.0
535592,201810021012,14,4,-2.0,ダ1700m,8/26(日),小倉,牝,16頭,ディヴァインハイツ,53.0,△川又,27.5,7.0,12R,３歳上５００万下,15,栗東,石坂正,468.0,2018.0


In [9]:
#欠損値の確認
keiba_data.isnull().sum()

race_num         0
horse_num        0
age              0
c_weight      1219
course           0
date             0
field            0
gender           0
head_count       0
horse_name       0
j_weight        43
jackie           0
odds          1961
popu          1961
race             0
race_name      230
rank             0
trainerA         0
trainerB         0
weight        1041
year             0
dtype: int64

In [10]:
# 欠損値処理
# 数値変数について、c_weightは競馬では基本０なので０で埋め、j_weight,weight,は平均値で埋め、odds,popu,race_nameはdropする。

In [11]:
#欠損値処理
keiba_data["c_weight"].fillna(0,inplace=True)
keiba_data["j_weight"].fillna(keiba_data["j_weight"].mean(),inplace=True)
keiba_data["weight"].fillna(keiba_data["weight"].mean(),inplace=True)
keiba_data.dropna(subset=["odds"],inplace=True)
keiba_data.dropna(subset=["popu"],inplace=True)
keiba_data.dropna(subset=["race_name"],inplace=True)

In [12]:
# データ型の確認
keiba_data.dtypes

race_num        int64
horse_num       int64
age             int64
c_weight      float64
course         object
date           object
field          object
gender         object
head_count     object
horse_name     object
j_weight      float64
jackie         object
odds          float64
popu          float64
race           object
race_name      object
rank           object
trainerA       object
trainerB       object
weight        float64
year          float64
dtype: object

In [13]:
#labelencoderを使って、カテゴリ変数を変換。
le=LabelEncoder()
keiba_categorical = keiba_data[["gender","field","horse_name","course","head_count","trainerA","trainerB","race","jackie","race_name"]].apply(le.fit_transform)
keiba_categorical = keiba_categorical.rename(columns={"race_name":"race_name_c","filed":"field_c","gender":"gender_c","horse_name":"horse_name_c","course":"course_c","head_count":"head_count_c","trainerA":"trainerA_c","trainerB":"trainerB_c","jackie":"jackie_c"})
keiba_data = pd.concat([keiba_data,keiba_categorical],axis=1)
# 変換前と不要な列を削除
keiba_data.drop(columns=["race_num","horse_num","date","year","race_name","race","trainerA","trainerB","course","field","gender","jackie","head_count","horse_name"],inplace=True)

In [14]:
# 特徴量生成
# １つ目はにoddsとpopuの積
# ２つ目は前回の体重
keiba_data["odds_popu"]=keiba_data["odds"]*keiba_data["popu"]
keiba_data["pre_weight"]=keiba_data["weight"]-keiba_data["c_weight"]

In [15]:
# rankの確認
keiba_data["rank"].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '中止', '失格', '17', '18'], dtype=object)

In [16]:
#中止、失格の数をそれぞれ確認。

a=keiba_data["rank"][keiba_data["rank"]=="中止"].count()
b=keiba_data["rank"][keiba_data["rank"]=="失格"].count()
print(a,b)

2680 24


In [17]:
#中止、失格の行は全て削除する。

delete_index = keiba_data.index[(keiba_data["rank"]=="中止") | (keiba_data["rank"]=="失格")]
keiba_data.drop(delete_index,inplace=True)

In [18]:
# データ型の確認
keiba_data.dtypes

age               int64
c_weight        float64
j_weight        float64
odds            float64
popu            float64
rank             object
weight          float64
gender_c          int64
horse_name_c      int64
course_c          int64
head_count_c      int64
trainerA_c        int64
trainerB_c        int64
jackie_c          int64
race_name_c       int64
odds_popu       float64
pre_weight      float64
dtype: object

In [19]:
# 1,2,3着かそれ以外かを分割して、２値分類問題にする。
keiba_data["rank"]=keiba_data["rank"].astype(int)
keiba_data = keiba_data.assign(target = (keiba_data['rank'] <= 3).astype(int))

In [20]:
keiba_data.head(10)

,age,c_weight,j_weight,odds,popu,rank,weight,gender_c,horse_name_c,course_c,head_count_c,trainerA_c,trainerB_c,jackie_c,race_name_c,odds_popu,pre_weight,target
0,2,0.0,54.0,4.3,2.0,1,468.0,1,30089,17,4,4,621,379,1461,8.6,468.0,1
1,2,0.0,54.0,5.2,4.0,2,460.0,2,191,17,4,2,426,699,1461,20.8,460.0,1
2,2,2.0,54.0,3.1,1.0,3,442.0,2,4764,17,4,4,165,407,1461,3.1,440.0,1
3,2,-4.0,54.0,23.0,8.0,4,482.0,2,56823,17,4,2,327,428,1461,184.0,486.0,0
4,2,-20.0,54.0,21.9,6.0,5,484.0,1,52788,17,4,2,446,739,1461,131.4,504.0,0
5,2,-4.0,54.0,4.6,3.0,6,452.0,2,33285,17,4,2,739,586,1461,13.8,456.0,0
6,2,-10.0,52.0,10.4,5.0,7,464.0,2,51466,17,4,4,695,75,1461,52.0,474.0,0
7,2,-6.0,54.0,154.0,12.0,8,446.0,2,45819,17,4,4,212,401,1461,1848.0,452.0,0
8,2,-8.0,53.0,111.7,11.0,9,418.0,2,49754,17,4,4,828,134,1461,1228.7,426.0,0
9,2,2.0,54.0,22.9,7.0,10,474.0,2,24084,17,4,2,460,561,1461,160.3,472.0,0


In [21]:
# trainデータおよびtestデータの分割と特徴量および目的変数の分割
import lightgbm as lgb
X = keiba_data.drop(['rank','target'], axis=1)
y = keiba_data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

#データの変換
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test)

In [ ]:
# ハイパーパラメータをoptunaで自動設定

%%time
!pip install optuna
from optuna.integration import lightgbm as lgb

params = {
    'objective': 'binary',
    'metric': 'auc'
}

best_params, history = {}, []
model = lgb.train(params, lgb_train, valid_sets=[lgb_train,lgb_eval],
                    verbose_eval=False,
                    num_boost_round=10,
                    early_stopping_rounds=10)
best_params_ = model.params

     |████████████████████████████████| 266kB 9.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 163kB 13.1MB/s 
     |████████████████████████████████| 133kB 14.9MB/s 
     |████████████████████████████████| 112kB 16.0MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp36-none-any.whl size=359761 sha256=fb5e080ce1945a3ff9cbb02d400717594220d00c7de5e927322397f3f492495d
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64b0ec6b964f89c0695a9dc6db6f851d0b54c5381a5c9cadfb
Successfully built optuna
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=f9fb329416b2641c50b8fea4fbed8b4472e89113cd9ce9a010a3f9fdebfb763a
  Stored in directory: /root/.cache/pip/whe

[I 2021-01-07 12:07:38,071] A new study created in memory with name: no-name-49b3c391-cb16-42c2-9278-a62b2082cef0
regularization_factors, val_score: 0.816124:  90%|######### | 18/20 [00:38<00:04,  2.14s/it]

In [ ]:
best_params_

In [ ]:
# モデルの作成
import lightgbm as lgb_orig

model = lgb_orig.train(best_params_, 
                       lgb_train,
                       valid_sets=lgb_eval,
                       num_boost_round=100,
                       early_stopping_rounds=10)

In [ ]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

y_pred = model.predict(X_test)

fpr, tpr, thresholds = roc_curve(y_test, y_pred)

plt.plot(fpr, tpr, marker='o')
plt.xlabel('FPR: False positive rate')
plt.ylabel('TPR: True positive rate')
plt.grid()

In [ ]:
#特徴量の重要度を表示。
keiba_data.drop(columns = ["rank","target"],inplace=True)
importance = pd.DataFrame(model.feature_importance(), index=keiba_data.columns,columns=['importance'])
importance=importance.sort_values(by="importance",ascending=False)
display(importance)

In [ ]:
# モデルの保存
import os
save_name = 'model_lgbm.pickle'
save_dir = '/content/drive/My Drive/keiba/models/'
os.makedirs(save_dir, exist_ok=True)
save_dir = '/content/drive/My Drive/keiba/models/' + save_name

pd.to_pickle(model,save_dir)